In [121]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from collections import Counter

for k in range(1, 4):
    df = pd.read_csv('dataset/breast-cancer-wisconsin/breast-cancer-wisconsin.csv')

    X = df.drop('diagnosis',axis=1)

    y=df['diagnosis']

    feature_names = X.columns

    le = LabelEncoder()
    y = le.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    transform = StandardScaler()
    X_train = transform.fit_transform(X_train)
    X_test = transform.transform(X_test)

    model = joblib.load('saved_model/breast-cancer-wisconsin/MLP.pkl')

    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    filename_explain = 'explain_set_lower/breast-cancer-wisconsin/MLP/lore/lore_explain_'+str(k)+'.csv'
    df = pd.read_csv(filename_explain)
    X_explain = df.drop('diagnosis',axis=1)
    y_explain=df['diagnosis']

    X_explain = transform.transform(X_explain)

    y_pred_explain = model.predict(X_explain)
    #print(y_pred_explain)

    y_pred_proba_explain = model.predict_proba(X_explain)
    compare = np.array(y_pred == y_pred_explain)
    
    #Only consider category RF SVC 0 or MLP 1
    #lower RF 1 SVC 0 or MLP 0
    M_index = np.where(y_pred==0)
    compare_M = np.array(y_pred[M_index] == y_pred_explain[M_index])
    counter = Counter(compare_M)
    dictcounter = dict(counter)
    if False in dictcounter:
        VAL=dictcounter[False]/sum(counter.values())
    else:
        VAL=0
    ConsistencyScores = 1-VAL
    print("k:",k)
    print("ConsistencyScores:",ConsistencyScores)
    print("VAL:",VAL)

k: 1
ConsistencyScores: 0.8411214953271028
VAL: 0.1588785046728972
k: 2
ConsistencyScores: 0.7102803738317758
VAL: 0.2897196261682243
k: 3
ConsistencyScores: 0.6261682242990654
VAL: 0.37383177570093457


In [122]:
k

3

In [123]:
test_data = pd.read_csv('dataset/breast-cancer-wisconsin/test_data.csv')
fail_index = np.where(compare_M==True)
M_index = np.array(M_index)
M_index = M_index.flatten()
fail = test_data.loc[M_index[fail_index]]
fail

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0,12.47,18.60,81.09,481.9,0.09965,0.10580,0.080050,0.038210,0.1925,...,14.970,24.64,96.05,677.9,0.14260,0.23780,0.26710,0.10150,0.3014,0.08750
3,0,12.40,17.68,81.47,467.8,0.10540,0.13160,0.077410,0.027990,0.1811,...,12.880,22.91,89.61,515.8,0.14500,0.26290,0.24030,0.07370,0.2556,0.09359
12,0,14.97,19.76,95.50,690.2,0.08421,0.05352,0.019470,0.019390,0.1515,...,15.980,25.82,102.30,782.1,0.10450,0.09995,0.07750,0.05754,0.2646,0.06085
14,0,12.34,12.27,78.94,468.5,0.09003,0.06307,0.029580,0.026470,0.1689,...,13.610,19.27,87.22,564.9,0.12920,0.20740,0.17910,0.10700,0.3110,0.07592
16,0,14.81,14.70,94.66,680.7,0.08472,0.05016,0.034160,0.025410,0.1659,...,15.610,17.58,101.70,760.2,0.11390,0.10110,0.11010,0.07955,0.2334,0.06142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,0,11.13,16.62,70.47,381.1,0.08151,0.03834,0.013690,0.013700,0.1511,...,11.680,20.29,74.35,421.1,0.10300,0.06219,0.04580,0.04044,0.2383,0.07083
160,0,9.72,18.22,60.73,288.1,0.06950,0.02344,0.000000,0.000000,0.1653,...,9.968,20.83,62.25,303.8,0.07117,0.02729,0.00000,0.00000,0.1909,0.06559
162,0,10.08,15.11,63.76,317.5,0.09267,0.04695,0.001597,0.002404,0.1703,...,11.870,21.18,75.39,437.0,0.15210,0.10190,0.00692,0.01042,0.2933,0.07697
163,0,12.56,19.07,81.92,485.8,0.08760,0.10380,0.103000,0.043910,0.1533,...,13.370,22.43,89.02,547.4,0.10960,0.20020,0.23880,0.09265,0.2121,0.07188


In [124]:
import os
file_path = 'dataset/breast-cancer-wisconsin/test_case/test_case_mlp_lower.csv'
file_exists = os.path.isfile(file_path)
if not file_exists:
    fail.to_csv(file_path, index=False, header=True)
else:
    fail.to_csv(file_path, mode='a', index=False, header=False)